# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### 1. Import required libraries

In [12]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

ModuleNotFoundError: No module named 'numpy'

### 2. Load the dataset

In [1]:
df = pd.read_csv('car+data.csv')

ModuleNotFoundError: No module named 'pandas'

### 3. Check the shape and basic information of the dataset.

In [ ]:
df.head()
df.shape
df.info()

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [ ]:
def check_duplicate(df):
    duplicate=df[df.duplicated()]
    return duplicate

duplicate=check_duplicate(df)
duplicate
df.drop('Car_Name',axis=1,inplace=True)

### 5. Drop the columns which you think redundant for the analysis.

In [ ]:
df.drop(['Seller_Type','Owner'], axis=1)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [ ]:
current_year = datetime.datetime.now().Year
df['age_of_the_car'] = current_year - df['Year']
df.drop('Year', axis=1)

### 7. Encode the categorical columns

In [ ]:
df[['Fuel_Type','Seller_Type','Transmission']] = df[['Fuel_Type','Seller_Type','Transmission']].apply(LabelEncoder().fit_transform)

### 8. Separate the target and independent features.

In [ ]:
target = df['Selling_Price']
independent_features = df.drop(['Seller_Type','Owner'], axis=1)

### 9. Split the data into train and test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    Car_Name, Selling_Price, test_size=0.2, random_state=42)

### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [ ]:
random_forest_regressor = RandomForestRegressor(random_state=42)
random_forest_regressor.fit(X_train, y_train)
y_train_pred = random_forest_regressor.predict(X_train)
y_test_pred = random_forest_regressor.predict(X_test)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
print("R2 score for training set:", r2_train)
print("R2 score for testing set:", r2_test)

### 11. Create a pickle file with an extension as .pkl

In [ ]:
data_to_pickle = {'example_list': [1, 2, 3], 'example_dict': {'a': 1, 'b': 2}}
pickle_file_path = 'model.pkl'
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(data_to_pickle, pickle_file)

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Car Price Prediction</title>
</head>
<body>
    <h1>Car Price Prediction</h1>
    <form id="predictionForm">
        <label for="year">Car Year:</label>
        <input type="number" id="year" name="year"><br><br>

        <!-- Add other input fields for features as needed -->

        <button type="button" onclick="predict()">Predict</button>
    </form>

    <p id="predictionResult"></p>

    <script>
        function predict() {
            const year = parseInt(document.getElementById('year').value);

            // Create a JSON object with the features
            const features = {
                'year': year
                // Add other features as needed
            };

            // Send a POST request to the Flask server for prediction
            fetch('/predict', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ 'features': features })
            })
            .then(response => response.json())
            .then(data => {
                const predictionResult = document.getElementById('predictionResult');
                predictionResult.innerHTML = 'Predicted price: ' + data.prediction;
            })
            .catch(error => console.error('Error:', error));
        }
    </script>
</body>
</html>

### b) Create app.py file and write the predict function

In [ ]:
def load_model():
    with open('model.pkl', 'rb') as model_file:
        model = pickle.load(model_file)
    return model

app = Flask(__name__)

model = load_model()

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json(force=True)
        features = data['features']
        prediction = model.predict([features])
        response = {
            'prediction': prediction.tolist()  
        }
        return jsonify(response)
    except Exception as e:
        return jsonify({'error': str(e)}), 400
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

### Happy Learning :)